<h1> Aplicação do SVD ao dataset em português do Word2Vec </h1>
<h3> O objetivo desse trabalho é compreender como o a técnica de Word2Vec funciona e utilizar o modelo de SVD para reduzir as dimensões do dataset de Word2Vec disponibilizado pelo Laboratório de Computação e Linguística da USP.</h3>
<h4><ol>
    <li>Introdução</li>
    <li>Objetivo</li>
    <li>Metodologia</li>
    <li>Estudo de Caso</li>
    <li>Conclusões</li>
    <li>Referências</li>
</ol></h4>

<h4>1. Introdução</h4>

<p>Como definir que uma frase ou uma palavra é similar a outra? Para resolver esse problema uma das soluções utilizadas é o treinamento de uma Rede Neural que usa como base uma representação vetorial das palavras. Isso pode ser observado nesse artigo bem explicativo do Luiza Labs [1].</p>

<p>Contudo, esse dataset de vetores de palavras (Word2Vec) em geral tem uma alta dimensionalidade, mais de 300 dimensões. Nesse trabalho deseja-se utilizar o método do SVD [2], para reduzir essa dimensionalidade preservando os "agrupamentos" encontrados após o treinamento.</p>


<h4>2. Objetivo</h4>

<p> Com o uso do SVD, reduzir o dataset de 300 para 150, 50 e 2 dimensões. Verificar se ainda são preservadas as principais características, em especial algumas operações conhecidas como:</p>

<p><i>Rei - Homem + Mulher = Rainha</i></p>

<h4>3. Metodologia</h4>

<ul>
    <li><p>Usar o código de SVD do Pacote Sci-Kit Learn [3] </p></li>
    <li><p>Uso do Modelo Pré-Treinado de Word2Vec [4] </p></li>
    <li><p>Aplicação de Visualização [5] </p></li>

</ul>

<h5>3.1 Representação Vetorial</h5>

<h4>Estudo de Caso</h4>

<p>Uso do Modelo Pré-Treinado de Word2Vec para português disponibilizado e treinado utilizando o algoritmo Skip-Gram</p>





In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gensim

In [10]:
# import data
import os
dirname = os.path.realpath('..')
#
filename = dirname+'/atividadeiv_SVD2Word2Vec/word2vec-pt-br/exemplo/wiki.pt.trigram.vector'
#filename = dirname+'/atividadeiv_SVD2Word2Vec/word2vec-pt-br/exemplo/skip_s300.txt'

model = gensim.models.KeyedVectors.load_word2vec_format(filename,binary=True)

In [11]:
# apply SVD

model.init_sims(replace=False)
#model.init_sims(replace=True)


result = model.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

#model.wv.syn0  # Input Embedding Matrix
#model.syn1     # Output embedding //with hierarchical softmax (hs=1)//
#model.syn1neg  # Output embedding //when it uses negative sampling (negative>0)//

print(model.wv.syn0)


#U, S, Vt = np.linalg.svd(C)

MemoryError: 

<h4>Referências</h4>

[1] https://medium.com/luizalabs/similaridade-entre-t%C3%ADtulos-de-produtos-com-word2vec-5e26199862f0

[2] 

[3] https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.linalg.svd.html

[4] https://github.com/felipeparpinelli/word2vec-pt-br